In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import numpy as np
import os
import datetime
from dotenv import load_dotenv
from quapopt import ancillary_functions as anf
from quapopt.circuits import backend_utilities as bck_utils
os.makedirs('../temp', exist_ok=True)

load_dotenv();

### Generate a random Hamiltonian instance

* Like in previous notebooks, we generate a random Hamiltonian instance.



In [13]:
from quapopt.hamiltonians.generators import build_hamiltonian_generator
from quapopt.data_analysis.data_handling import (CoefficientsType,
                                                 CoefficientsDistribution,
                                                 CoefficientsDistributionSpecifier,
                                                 HamiltonianModels)

number_of_qubits = 10
seed_cost_hamiltonian = 1

coefficients_type = CoefficientsType.CONTINUOUS
coefficients_distribution = CoefficientsDistribution.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}
coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

# We generate a Hamiltonian instance. In this case it's a random Sherrington-Kirkpatrick Hamiltonian
hamiltonian_model = HamiltonianModels.SherringtonKirkpatrick
localities = (2,)
generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier)

cost_hamiltonian = generator_cost_hamiltonian.generate_instance(number_of_qubits=number_of_qubits,
                                                                seed=seed_cost_hamiltonian,
                                                                read_from_drive_if_present=True)

print("Class description (cost):", cost_hamiltonian.hamiltonian_class_description)
print("Instance description (cost):", cost_hamiltonian.hamiltonian_instance_description)

if cost_hamiltonian.lowest_energy is None:
    print("SOLVING THE HAMILTONIAN CLASSICALLY")
    # if we wish, we can solve the Hamiltonian classically
    cost_hamiltonian.solve_hamiltonian(both_directions=True)

ground_state_energy = cost_hamiltonian.ground_state_energy
highest_energy = cost_hamiltonian.highest_energy

File not found!
FILE NOT FOUND!
Class description (cost): HMN=SK;LOC=(2,);CFD=CT~CON_CDN~UNI_CDP~low~-1_high~1_step~1
Instance description (cost): NOQ=10;HII=1
SOLVING THE HAMILTONIAN CLASSICALLY


### Wrapper for QAOA Sampler
* In the context of QAOA, we can wrap the simulator a bit more so it's a part of more abstract framework useful in further optimization.
* The "QAOARunnerSampler" is an abstract class that can be used to run the QAOA with different backends (here we will be interested mostly in qiskit though).
* The optimization is a bit more general than usual -- a) we can optimize over ansatz for which phase is not the same as cost; b) we can optimize over multiple representations of the Hamiltonian (e.g., permutations or bitflip gauges); note that b) requires classical optimizer that can actually support it (notable example: optuna with categorical variables).
* Here we will run p=1 QAOA so we can visualize the landscapes nicely.
* Note: if you run into some errors regarding import of "cython" modules, this means you haven't run 'pixi run build_cpp' command properly.


In [14]:
from quapopt.optimization.QAOA.implementation.QAOARunnerSampler import QAOARunnerSampler
from quapopt.data_analysis.data_handling import LoggingLevel

#Specify some kwargs for logging
# (this is generally not required, but here we want to log the results in a specific folder)
#Experiment set id is used to identify results in the database later. (note: here 'database' means just a bunch of hierarchical folders, mainly with .csv files)
experiment_set_id = f"{anf.create_random_uuid()}"

logging_level = LoggingLevel.DETAILED

#The logger will create subfolders specified here
logger_kwargs_main = {'experiment_folders_hierarchy': ['SimulationResults',
                                                       'TestingQAOAOptimization'],
                      'experiment_set_id': experiment_set_id,  #Used to group the experiments
                      }

print(logger_kwargs_main)



{'experiment_folders_hierarchy': ['SimulationResults', 'TestingQAOAOptimization'], 'experiment_set_id': '8f80362cd67c4ffd8e92e2bc253d78a2'}


In [15]:
sampler_backend = 'Qiskit'

if sampler_backend.lower() == 'qiskit':
    from quapopt.optimization.QAOA import QubitMappingType
    #Here we assume that IBM's account data is in the environment variables. This is read from .env file in the root of the repo.
    #You can also just set the variable values here.
    #NOTE: not all of those variables are needed for setting up the account. Please see the QiskitRuntimeService for details
    ibm_credentials_path = os.getenv('IBM_CREDENTIALS_PATH')
    ibm_account_name = os.getenv('IBM_ACCOUNT_NAME')
    ibm_instance = os.getenv('IBM_INSTANCE_NAME')
    ibm_token = os.getenv('IBM_TOKEN')
    ibm_channel = os.getenv('IBM_CHANNEL')

    #get the provider for the IBM Qiskit backend; we need it to run the SABRE pass manager and to run on actual hardware
    provider_ibm = bck_utils.get_qiskit_provider(
        credentials_path=ibm_credentials_path,
        instance_ibm=ibm_instance,
        account_name=ibm_account_name,
        token=ibm_token,
        channel=ibm_channel,

    )
    #get a default simulator backend and pass manager for the SABRE pass manager
    #This can be changed if desired
    qiskit_backend, qiskit_pass_manager, pass_manager_kwargs = bck_utils.get_default_qiskit_backend_and_pass_manager(
        provider=provider_ibm,
        backend_name='aer',  #might change to actual backend for simulation for real device
        qubit_mapping_type=QubitMappingType.sabre,
        backend_kwargs=None,  #use defaults
        pass_manager_kwargs=None,  #use defaults
    )

    qiskit_sampler = bck_utils.create_qiskit_sampler(qiskit_backend=qiskit_backend,
                                                     simulation=True,
                                                     qiskit_sampler_options=None,  #use defaults
                                                     session_ibm=None,  #add if desired
                                                     override_to_noiseless_simulation=True,
                                                     #remove default device's noise model
                                                     )




In [16]:
logger_kwargs_sampler = {**logger_kwargs_main,
                         **{'experiment_instance_id': f"IdealSampling{sampler_backend.upper()}"}}

numpy_rng_sampling = np.random.default_rng(seed=0)
#Set up the raw sampler with Hamiltonians as an input
qaoa_sampler = QAOARunnerSampler(
    hamiltonian_representations_cost=[cost_hamiltonian],
    hamiltonian_representations_phase=None,
    store_n_best_results=1,
    store_full_information_in_history=False,
    numpy_rng_sampling=numpy_rng_sampling,
    logger_kwargs=logger_kwargs_sampler,
    logging_level=logging_level
)

#Here we basically run the same thing as before but inside the class;
#so the ansatze are set up for possibly multiple cost Hamiltonians
if sampler_backend.lower() == 'qiskit':
    qaoa_sampler.initialize_backend_qiskit(simulation=True,
                                           qiskit_backend=qiskit_backend,
                                           noiseless_simulation=True,
                                           qiskit_pass_manager=qiskit_pass_manager,
                                           qaoa_depth=1,
                                           qubit_indices_physical=None,
                                           classical_indices=None)
elif sampler_backend.lower() == 'qokit':
    #If we have GPU, we use qokit for demonstration because it's faster than qiskit
    qaoa_sampler.initialize_backend_qokit(qokit_backend='auto')

elif sampler_backend.lower() == 'python':
    qaoa_sampler.initialize_backend_python(backend='auto')

else:
    raise NotImplementedError("Only qiskit and qokit are supported")

#print(qaoa_sampler._backends[0].ansatz_circuit)

No existing metadata found for the specified experiment set. 


### Wrapper for optimizers

* In general, we will want to use our QAOA samplers with some optimizers.
* We can run:
1. Optuna -- a hyperparameter optimization framework. It has some nice optimizers and visualization features.
2. Custom grid sampling - we can run a simple grid sampling to get a feel for the landscape.
3. Scipy -- we can also use scipy's optimizers.




In [17]:
from quapopt.optimization.parameter_setting.variational.QAOAOptimizationRunner import QAOAOptimizationRunner
from quapopt.optimization.parameter_setting.variational.scipy_tools.ScipyOptimizerWrapped import ScipyOptimizerWrapped

#we can specify the details of classical optimizer here.
classical_optimizer = ScipyOptimizerWrapped(parameters_bounds=[(-np.pi, np.pi)] * 2, #
                                            #argument_names = ['Angles-0', 'Angles-1'],
                                            optimizer_name='COBYQA',
                                            optimizer_kwargs=None,
                                            basinhopping=True,
                                            basinhopping_kwargs={'niter': 1},
                                            starting_point=[0.05] * 2
                                            )

qaoa_optimizer = QAOAOptimizationRunner(qaoa_runner=qaoa_sampler)

In [18]:
#Number of objective function calls
number_of_function_calls = 200
#Number of measurements to estimate the expectation values
number_of_samples = 10 ** 3

best_results_noiseless, optimization_results_noiseless = qaoa_optimizer.run_optimization(qaoa_depth=1,
                                                                                         number_of_function_calls=number_of_function_calls,
                                                                                         number_of_samples=number_of_samples,
                                                                                         classical_optimizer=classical_optimizer,
                                                                                         optimizer_seed=42,
                                                                                         #Note that we can add readout noise here
                                                                                         measurement_noise=None,
                                                                                         verbosity=0,
                                                                                         show_progress_bar=True)



COBYQA:   0%|          | 0/200 [00:00<?, ?it/s]

In [19]:
#note: "best" result here means the one containing the best BITSTRING, not necessary the best expected value!
best_result_noiseless = best_results_noiseless[0]
best_energy_bts_noiseless = best_result_noiseless[0]
best_bts_noiseless = best_result_noiseless[1][0]
best_metadata_noiseless = best_result_noiseless[1][-1]

print("BEST ENERGY (BTS, NOISELESS):", best_energy_bts_noiseless)
print("BEST BTS (NOISELESS):", best_bts_noiseless)
print('BEST MEAN ENERGY (NOISELESS):', optimization_results_noiseless.best_value)
best_metadata_noiseless.to_dataframe_main()

BEST ENERGY (BTS, NOISELESS): -11.925219831988215
BEST BTS (NOISELESS): (np.int32(0), np.int32(1), np.int32(1), np.int32(0), np.int32(1), np.int32(1), np.int32(0), np.int32(1), np.int32(0), np.int32(1))
BEST MEAN ENERGY (NOISELESS): -5.464043519452214


,TrialIndex,HamiltonianRepresentationIndex,Angles,EnergyMean,EnergyBest,BitstringBest
0,0,0,"[0.05, 0.05]",0.434445,-11.92522,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]"


### Add measurement noise

* We can add classical measurement noise to the simulation. Here we will emulate amplitude damping.

In [20]:
from quapopt.circuits.noise.simulation.ClassicalMeasurementNoiseSampler import ClassicalMeasurementNoiseSampler, \
    MeasurementNoiseType

# Fully asymmetric noise -- equivalent to amplitude damping at the end of the circuit
p_01 = 0.5
p_10 = None
CMNS = ClassicalMeasurementNoiseSampler(noise_type=MeasurementNoiseType.TP_1q_identical,
                                        noise_description={'p_01': p_01,
                                                           'p_10': p_10})

logger_kwargs_sampler_noisy = {**logger_kwargs_main,
                               **{'experiment_instance_id': f"NoisySampling{sampler_backend.upper()}"}}
qaoa_optimizer.qaoa_runner.reinitialize_logger(**logger_kwargs_sampler_noisy)

best_results_noisy, optimization_results_noisy = qaoa_optimizer.run_optimization(qaoa_depth=1,
                                                                                 number_of_function_calls=number_of_function_calls,
                                                                                 number_of_samples=number_of_samples,
                                                                                 classical_optimizer=classical_optimizer,
                                                                                 optimizer_seed=42,
                                                                                 #Note that are adding noise here
                                                                                 measurement_noise=CMNS,
                                                                                 verbosity=0,
                                                                                 show_progress_bar=True
                                                                                 )


COBYQA:   0%|          | 0/200 [00:00<?, ?it/s]

Starting the optimization procedure.
Initial trust-region radius: 1.0.
Final trust-region radius: 1e-06.
Maximum number of function evaluations: 200.
Maximum number of iterations: 2000.

__objective_function([ 5.000e-02  5.000e-02]) = 0.6076709914542735
__objective_function([ 1.050e+00  5.000e-02]) = 0.3989073995575309
__objective_function([ 5.000e-02  1.050e+00]) = -0.0021925749145448206
__objective_function([-9.500e-01  5.000e-02]) = 0.4426128734126687
__objective_function([ 5.000e-02 -9.500e-01]) = 0.5825974084138871
__objective_function([ 7.356e-02  2.050e+00]) = 0.6232116398066282

New trust-region radius: 0.1.
Number of function evaluations: 6.
Number of iterations: 1.
Least value of __objective_function: -0.0021925749145448206.
Maximum constraint violation: 0.0.
Corresponding point: [ 5.000e-02  1.050e+00].

__objective_function([-4.499e-01  1.042e+00]) = 1.4197135357819497
__objective_function([ 3.000e-01  1.054e+00]) = -0.7254890056401491
__objective_function([ 5.452e-01  1.10

In [21]:
best_result_noisy = best_results_noisy[0]

best_energy_bts_noisy = best_result_noisy[0]
best_bts_noisy = best_result_noisy[1][0]
best_metadata_noisy = best_result_noisy[1][-1]

print("BEST ENERGY (BTS, NOISELESS):", best_energy_bts_noiseless)
print("BEST BTS (NOISELESS):", best_bts_noiseless)
print('BEST MEAN ENERGY (NOISELESS):', optimization_results_noiseless.best_value)
print("____________")
print("BEST ENERGY (BTS, NOISY):", best_energy_bts_noisy)
print("BEST BTS:", best_bts_noisy)
print('BEST MEAN ENERGY (NOISY):', optimization_results_noisy.best_value)

BEST ENERGY (BTS, NOISELESS): -11.925219831988215
BEST BTS (NOISELESS): (np.int32(0), np.int32(1), np.int32(1), np.int32(0), np.int32(1), np.int32(1), np.int32(0), np.int32(1), np.int32(0), np.int32(1))
BEST MEAN ENERGY (NOISELESS): -5.464043519452214
____________
BEST ENERGY (BTS, NOISY): -11.925219831988215
BEST BTS: (np.int32(1), np.int32(0), np.int32(0), np.int32(1), np.int32(0), np.int32(0), np.int32(1), np.int32(0), np.int32(1), np.int32(0))
BEST MEAN ENERGY (NOISY): -1.2035657596066593


### Reading data

* In a moment, we will want to visualize data.
* We could take the data directly from the output of optimization, but here we will read it from database to test the logging:
1. We need to set up the logger for the results. (the optimizer does it automatically, but we are pretending we ran the optimization separately and now just reading the results).
2. The input requires: the cost Hamiltonian, and potentially the folder hierarchy and the main table name prefix so it can identify what data we want to read.
3. We can then read the data from the database and visualize it.

In [22]:
from quapopt.data_analysis.data_handling import ResultsLogger
from quapopt.data_analysis.data_handling import (STANDARD_NAMES_DATA_TYPES as SNDT,
                                                 STANDARD_NAMES_VARIABLES as SNV)

results_logger_sampler = ResultsLogger(**logger_kwargs_sampler)

df_noiseless = results_logger_sampler.read_results(data_type=SNDT.OptimizationOverview,
                                                   experiment_instance_ids=['IdealSamplingQISKIT'])
df_noiseless_direct = optimization_results_noiseless.trials_dataframe

#the same logger can be used; instance_ids are used for default writing, but reading is more flexible.
#Both writing and reading filter by experiment_set_id by default
df_noisy = results_logger_sampler.read_results(data_type=SNDT.OptimizationOverview,
                                               experiment_instance_ids=['NoisySamplingQISKIT'])
df_noisy_direct = optimization_results_noisy.trials_dataframe


In [23]:
df_noiseless

,TrialIndex,HamiltonianRepresentationIndex,Angles,EnergyMean,EnergyBest,BitstringBest,ExperimentInstanceID
0,0,0,"[0.05, 0.05]",0.434445,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",IdealSamplingQISKIT
1,1,0,"[1.05, 0.05]",-0.071480,-10.400989,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]",IdealSamplingQISKIT
2,2,0,"[0.05, 1.05]",-1.131064,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",IdealSamplingQISKIT
3,3,0,"[-0.95, 0.05]",-0.049302,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",IdealSamplingQISKIT
4,4,0,"[0.05, -0.95]",0.985562,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",IdealSamplingQISKIT
...,...,...,...,...,...,...,...
99,99,0,"[0.31089935128146867, 2.7487685314253065]",-5.464044,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",IdealSamplingQISKIT
100,100,0,"[0.310898145865892, 2.748770251731043]",-5.464044,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",IdealSamplingQISKIT
101,101,0,"[0.3108976079054741, 2.748767392935578]",-5.464044,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",IdealSamplingQISKIT
102,102,0,"[0.31089618097363686, 2.748767385143974]",-5.464044,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",IdealSamplingQISKIT


In [24]:
df_noiseless_direct

,TrialIndex,FunctionValue,WallClockTime,Arguments
0,0,0.434445,0.074240,"[[0.05, 0.05]]"
1,1,-0.071480,0.063980,"[[1.05, 0.05]]"
2,2,-1.131064,0.068080,"[[0.05, 1.05]]"
3,3,-0.049302,0.073973,"[[-0.95, 0.05]]"
4,4,0.985562,0.067867,"[[0.05, -0.95]]"
...,...,...,...,...
99,99,-5.464044,0.031120,"[[0.31089935128146867, 2.7487685314253065]]"
100,100,-5.464044,0.032432,"[[0.310898145865892, 2.748770251731043]]"
101,101,-5.464044,0.039852,"[[0.3108976079054741, 2.748767392935578]]"
102,102,-5.464044,0.037239,"[[0.31089618097363686, 2.748767385143974]]"


In [25]:
df_noisy

,TrialIndex,HamiltonianRepresentationIndex,Angles,EnergyMean,EnergyBest,BitstringBest,ExperimentInstanceID
104,0,0,"[0.05, 0.05]",0.607671,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",NoisySamplingQISKIT
105,1,0,"[1.05, 0.05]",0.398907,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",NoisySamplingQISKIT
106,2,0,"[0.05, 1.05]",-0.002193,-10.400989,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]",NoisySamplingQISKIT
107,3,0,"[-0.95, 0.05]",0.442613,-10.400989,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]",NoisySamplingQISKIT
108,4,0,"[0.05, -0.95]",0.582597,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",NoisySamplingQISKIT
...,...,...,...,...,...,...,...
195,91,0,"[0.3130413915154838, 2.8029601369193347]",-0.816710,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",NoisySamplingQISKIT
196,92,0,"[0.3130392522882194, 2.8029616514378093]",-0.810660,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",NoisySamplingQISKIT
197,93,0,"[0.3130410493946055, 2.802962345508098]",-0.976788,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",NoisySamplingQISKIT
198,94,0,"[0.31304041187130727, 2.8029613152825514]",-0.916455,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",NoisySamplingQISKIT


In [26]:
df_noisy_direct

,TrialIndex,FunctionValue,WallClockTime,Arguments
0,0,0.607671,0.090759,"[[0.05, 0.05]]"
1,1,0.398907,0.074465,"[[1.05, 0.05]]"
2,2,-0.002193,0.076320,"[[0.05, 1.05]]"
3,3,0.442613,0.065866,"[[-0.95, 0.05]]"
4,4,0.582597,0.052799,"[[0.05, -0.95]]"
...,...,...,...,...
91,91,-0.816710,0.085721,"[[0.3130413915154838, 2.8029601369193347]]"
92,92,-0.810660,0.084423,"[[0.3130392522882194, 2.8029616514378093]]"
93,93,-0.976788,0.070625,"[[0.3130410493946055, 2.802962345508098]]"
94,94,-0.916455,0.074020,"[[0.31304041187130727, 2.8029613152825514]]"


### Visualization

* After running optimization, we can visualize it using various methods with plotly.


In [27]:
import plotly
from quapopt.data_analysis.visualization import optimization_visualization as opt_vis

plotly.io.templates.default = "plotly"
plotly.offline.init_notebook_mode(connected=True)

x_name = f"{SNV.Angles.id_long}-0"
y_name = f"{SNV.Angles.id_long}-1"
fom_name = f"{SNV.EnergyMean.id_long}"

for _df in [df_noiseless, df_noisy]:
    _df[x_name] = _df['Angles'].apply(lambda x: x[0])
    _df[y_name] = _df['Angles'].apply(lambda x: x[1])

In [28]:
trajectory_fig = opt_vis.plot_multiple_heatmaps(dataframes=[df_noiseless, df_noisy],
                                                x_name=x_name,
                                                y_name=y_name,
                                                fom_name=fom_name,
                                                bounds_x=(-np.pi, np.pi),
                                                bounds_y=(-np.pi, np.pi),
                                                add_trajectories=True,
                                                titles=['Ideal', 'Noisy'],
                                                global_normalization=True,
                                                colormap_name='Viridis',
                                                minimization=True)

plotly.offline.plot(trajectory_fig,
                    filename='../temp/trajectories_plot.html')
# trajectory_fig.show()


'../temp/trajectories_plot.html'

### Grid sampling
* We can also run a simple grid sampling to get a feel for the landscape and to compare it with the optimization results.
* Here we run bruteforce grid sampling that is not scalable for large dimensions
* We will use it to plot the background grid of points for the trajectories of the TPE optimizer.
* Note that this works only for p=1, otherwise it's hard to visualize

In [29]:
from quapopt.optimization.parameter_setting.non_adaptive_optimization.SimpleGridOptimizer import SimpleGridOptimizer
from quapopt.optimization.parameter_setting import ParametersBoundType as PBT
from quapopt.optimization.QAOA.simulation.QAOARunnerExpValues import QAOARunnerExpValues

grid_size_total = 10 ** 4

grid_sampler = SimpleGridOptimizer(parameter_bounds=[(PBT.RANGE, (-np.pi, np.pi)),
                                                     (PBT.RANGE, (-np.pi, np.pi))
                                                     ],
                                   max_trials=grid_size_total)

logger_kwargs_grid = {**logger_kwargs_main,
                      **{'experiment_instance_id': 'IdealExpValuesGrid'}}

#runner that calculates expected values for p=1 QAOA (NO SAMPLING)
qaoa_exp_values_simulator = QAOARunnerExpValues(
    hamiltonian_representations_cost=[cost_hamiltonian],
    store_full_information_in_history=False,
    simulator_name=None,
    logger_kwargs=logger_kwargs_grid,
    logging_level=None)

qaoa_optimizer_exp_values = QAOAOptimizationRunner(qaoa_runner=qaoa_exp_values_simulator)


In [30]:
#noiseless grid
best_result_grid_noiseless, opt_results_grid_noiseless = qaoa_optimizer_exp_values.run_optimization(
    qaoa_depth=1,
    number_of_function_calls=grid_size_total,
    classical_optimizer=grid_sampler,
    measurement_noise=None,
    memory_intensive=True, #for grid sampling, if True, this scales (almost) only with the number of distinct gamma angles by storing some intermediate results; but takes some memory for large system sizes
    show_progress_bar=True,
    verbosity=1,
    # analytical_betas=True
)


GridOptimizer:   0%|          | 0/10000 [00:00<?, ?it/s]

New minimum found at trial: 0 with function value: inf -> 1.8970323264925344e-16
New minimum found at trial: 12 with function value: 1.8970323264925344e-16 -> -0.07159083503163204
New minimum found at trial: 13 with function value: -0.07159083503163204 -> -0.16915375259296894
New minimum found at trial: 14 with function value: -0.16915375259296894 -> -0.25935579529291497
New minimum found at trial: 15 with function value: -0.25935579529291497 -> -0.33641477792100044
New minimum found at trial: 16 with function value: -0.33641477792100044 -> -0.39539101955261463
New minimum found at trial: 17 with function value: -0.39539101955261463 -> -0.43250398992564654
New minimum found at trial: 18 with function value: -0.43250398992564654 -> -0.4453746512491074
New minimum found at trial: 3803 with function value: -0.4453746512491074 -> -0.521279794070765
New minimum found at trial: 3804 with function value: -0.521279794070765 -> -0.6224219729369312
New minimum found at trial: 3805 with function 

In [31]:
#noisy grid (note: this is possible to calculate only for a simple uncorrelated classical measurement noise model)
best_result_grid_noisy, opt_results_grid_noisy = qaoa_optimizer_exp_values.run_optimization(qaoa_depth=1,
                                                                                            number_of_function_calls=grid_size_total,
                                                                                            classical_optimizer=grid_sampler,
                                                                                            memory_intensive=True,
                                                                                            verbosity=1,
                                                                                            measurement_noise=CMNS,
                                                                                            show_progress_bar=True
                                                                                            )

GridOptimizer:   0%|          | 0/10000 [00:00<?, ?it/s]

New minimum found at trial: 0 with function value: inf -> 0.37237026472161294
New minimum found at trial: 12 with function value: 0.37237026472161294 -> 0.35447255575193043
New minimum found at trial: 13 with function value: 0.35447255575193043 -> 0.33008182608653114
New minimum found at trial: 14 with function value: 0.33008182608653114 -> 0.3075313151575497
New minimum found at trial: 15 with function value: 0.3075313151575497 -> 0.2882665692838853
New minimum found at trial: 16 with function value: 0.2882665692838853 -> 0.273522508710578
New minimum found at trial: 17 with function value: 0.273522508710578 -> 0.26424426601375833
New minimum found at trial: 18 with function value: 0.26424426601375833 -> 0.26102660064781213
New minimum found at trial: 3803 with function value: 0.26102660064781213 -> 0.24205031585210757
New minimum found at trial: 3804 with function value: 0.24205031585210757 -> 0.2167647711156983
New minimum found at trial: 3805 with function value: 0.2167647711156983

In [34]:
#this can made it a bit slower than just opening in browser, but it allows to show the plots in Jupyter cells
show_plots_in_jupyter_cell = False

#TODO(FBM): this should be automated in the function, refactor
names_map = {f"FunctionValue": f"{SNV.EnergyMean.id_long}"}
df_noiseless_grid = opt_results_grid_noiseless.trials_dataframe
df_noiseless_grid.rename(columns=names_map, inplace=True)
df_noisy_grid = opt_results_grid_noisy.trials_dataframe
df_noisy_grid.rename(columns=names_map, inplace=True)
for _df in [df_noiseless_grid, df_noisy_grid]:
    if x_name not in _df.columns:
        _df[x_name] = _df['Arguments'].apply(lambda x:x[0])
    if y_name not in _df.columns:
        _df[y_name] = _df['Arguments'].apply(lambda x:x[1])

for plot_in_3d in [False, True]:
    heatmap_grid_noiseless, scatter_grid_noiseless = opt_vis.get_interpolated_heatmap(df=df_noiseless_grid,
                                                                                      x_name=x_name,
                                                                                      y_name=y_name,
                                                                                      fom_name=fom_name,
                                                                                      bounds_x=(-np.pi, np.pi),
                                                                                      bounds_y=(-np.pi, np.pi),
                                                                                      in_3d=plot_in_3d
                                                                                      )
    heatmap_grid_noisy, scatter_grid_noisy = opt_vis.get_interpolated_heatmap(df=df_noisy_grid,
                                                                              x_name=x_name,
                                                                              y_name=y_name,
                                                                              fom_name=fom_name,
                                                                              bounds_x=(-np.pi, np.pi),
                                                                              bounds_y=(-np.pi, np.pi),
                                                                              in_3d=plot_in_3d
                                                                              )

    trajectory_fig2 = opt_vis.plot_multiple_heatmaps(dataframes=[df_noiseless, df_noisy],
                                                     x_name=x_name,
                                                     y_name=y_name,
                                                     fom_name=fom_name,
                                                     bounds_x=(-np.pi, np.pi),
                                                     bounds_y=(-np.pi, np.pi),
                                                     heatmaps_input=[heatmap_grid_noiseless, heatmap_grid_noisy],
                                                     scatter_inputs=[scatter_grid_noiseless, scatter_grid_noisy],
                                                     add_trajectories=True,
                                                     titles=['Ideal', 'Noisy'],
                                                     suffixes=['(Ideal)', '(Noisy)'],
                                                     global_normalization=False,
                                                     colormap_name='Viridis',
                                                     minimization=True,
                                                     in_3d=plot_in_3d)

    plotly.offline.plot(trajectory_fig2,
                        filename=f'../temp/trajectories_grid_plot_3d-{plot_in_3d}.html')
    if not show_plots_in_jupyter_cell:
        continue
    trajectory_fig2.show()
